In [ ]:
from pymongo.mongo_client import MongoClient
import json
from tqdm import tqdm

In [ ]:
def get_mongo_client(mongo_uri):
    client = MongoClient(mongo_uri)
    return client

In [ ]:
# import requests
# import os
# from dotenv import load_dotenv, find_dotenv

# _ = load_dotenv(find_dotenv())

# API_KEY = os.getenv("OPENROUTER_KEY")
# url = "https://openrouter.ai/api/v1/key"
# headers = {
#     "Authorization": f"Bearer {API_KEY}"
# }
# resp = requests.get(url, headers=headers)
# resp_json = resp.json()
# data = resp_json["data"]
# usage = data["usage"]
# limit = data["limit"]
# if limit is not None:
#     remaining = limit - usage
# else:
#     remaining = None  # unlimited or no cap
# print("Used:", usage, "Limit:", limit, "Remaining:", remaining)

In [ ]:
mongo_client = get_mongo_client("mongodb://localhost:27018/?directConnection=true")
[db_name for db_name in mongo_client.list_database_names() if 'hotpot' in db_name]
[db_name for db_name in mongo_client.list_database_names() if ('hotpot' not in db_name) and ('triplets' in db_name)]


In [ ]:
db = mongo_client.get_database('onto_triplets_db_llama')
db.list_collection_names()

In [ ]:
result = db.get_collection("filtered_triplets").aggregate([
    {
        "$group": {
            "_id": {
                "$concat": [
                    "$sample_id",
                    "`_",
                    { "$toString": "$source_text_id" }
                ]
            },
            "count": {"$sum": 1}
        }
    }
])

filtered_triplets = {}
for doc in result:
    filtered_triplets[doc["_id"]] = doc["count"]

filtered_triplets
result = db.get_collection("ontology_filtered_triplets").aggregate([
    {
        "$group": {
            "_id": "$sample_id",
            "count": {"$sum": 1}
        }
    }
])

ontology_filtered_triplets = {}
for doc in result:
    ontology_filtered_triplets[doc["_id"]] = doc["count"]

ontology_filtered_triplets

In [ ]:
db.get_collection("initial_triplets").count_documents({})

In [ ]:
result = db.get_collection("initial_triplets").aggregate([
    # {
    #     "$group": {
    #         "_id": {
    #             "$concat": [
    #                 "$sample_id",
    #                 "_",
    #                 { "$toString": "$source_text_id" }
    #             ]
    #         },
    #         "count": {"$sum": 1}
    #     }
    # }
        {
        "$group": {
            "_id": "$sample_id",
            "count": {"$sum": 1}
        }
    }
])

initial_triplets = {}
for doc in result:
    initial_triplets[doc["_id"]] = doc["count"]

initial_triplets

In [ ]:
sum(initial_triplets.values()) 

In [ ]:
result = db.get_collection("triplets").aggregate([
    # {
    #     "$group": {
    #         "_id": {
    #             "$concat": [
    #                 "$sample_id",
    #                 "_",
    #                 { "$toString": "$source_text_id" }
    #             ]
    #         },
    #         "count": {"$sum": 1}
    #     }
    # }
        {
        "$group": {
            "_id": "$sample_id",
            "count": {"$sum": 1}
        }
    }
])

triplets = {}
for doc in result:
    triplets[doc["_id"]] = doc["count"]

triplets

In [ ]:
len(triplets)

In [ ]:
import numpy as np
ratios = []
for triplet in initial_triplets:
    filtered_triplets_count = 0
    triplets_count = 0

    if triplet in filtered_triplets:
        filtered_triplets_count = filtered_triplets[triplet]
    if triplet in triplets:
        triplets_count = triplets[triplet]  
        
    if not initial_triplets[triplet] == filtered_triplets_count + triplets_count:
        print(f"Triplet: {triplet}")
        print(f"Initial triplets count: {initial_triplets[triplet]}")
        print(f"Filtered triplets count: {filtered_triplets_count}")
        print(f"Triplets count: {triplets_count}")
        print(f"Ratio: {filtered_triplets_count / initial_triplets[triplet]}")
        print("-" * 100)
    
    ratios += [filtered_triplets_count / initial_triplets[triplet]]

np.mean(ratios), np.std(ratios)

In [ ]:
import numpy as np
ratios = []
for text_sample in initial_triplets:
    ontology_filtered_triplets_count = 0
    filtered_triplets_count = 0
    triplets_count = 0

    if text_sample in ontology_filtered_triplets:
        ontology_filtered_triplets_count = ontology_filtered_triplets[text_sample]
    if text_sample in filtered_triplets:
        filtered_triplets_count = filtered_triplets[text_sample]
    if text_sample in triplets:
        triplets_count = triplets[text_sample]  
        
    if initial_triplets[text_sample] == filtered_triplets_count + triplets_count + ontology_filtered_triplets_count:
        pass
        # print(f"Triplet: {triplet}")
        # print(f"Initial triplets count: {initial_triplets[text_sample]}")
        # print(f"Filtered triplets count: {filtered_triplets_count}")
        # print(f"Ontology filtered triplets count: {ontology_filtered_triplets_count}")
        # print(f"Triplets count: {triplets_count}")
        # print(f"Ratio: {ontology_filtered_triplets_count / initial_triplets[text_sample]}")
        # print("-" * 100)
    else:
        print(f"Triplet: {text_sample}")
        print(f"Initial triplets count: {initial_triplets[text_sample]}")
        print(f"Filtered triplets count: {filtered_triplets_count}")
        print(f"Ontology filtered triplets count: {ontology_filtered_triplets_count}")
        print(f"Triplets count: {triplets_count}")
        # print(f"Ratio: {ontology_filtered_triplets_count / (filtered_triplets_count + triplets_count + ontology_filtered_triplets_count)}")
        print("-" * 100)
    if (filtered_triplets_count + triplets_count + ontology_filtered_triplets_count) != 0:
        ratios += [ontology_filtered_triplets_count/ (filtered_triplets_count + triplets_count + ontology_filtered_triplets_count)]

np.mean(ratios), np.std(ratios)

## Token count

In [ ]:
mongo_client = get_mongo_client("mongodb://localhost:27018/?directConnection=true")
[db_name for db_name in mongo_client.list_database_names() if 'hotpot' in db_name]
# mongo_client.list_database_names()
[db_name for db_name in mongo_client.list_database_names() if ('hotpot' not in db_name) and ('triplets' in db_name)]


In [ ]:
db = mongo_client.get_database('musique_gpt4_1_mini_onto_triplets')
db.list_collection_names()

In [ ]:
db.get_collection("triplets").find_one({})

In [ ]:
pipeline = [
    {
        "$group": {
            "_id": {
                "sample_id": "$sample_id",
                "source_text_id": "$source_text_id"
            },
            "completion_token_num": {"$first": "$completion_token_num"},
            "prompt_token_num": {"$first": "$prompt_token_num"}
        }
    }
]

results = list(db.get_collection("initial_triplets").aggregate(pipeline))
initial_stage_prompt_token_sum = 0
initial_stage_completion_token_sum = 0
for doc in results:
    initial_stage_prompt_token_sum += doc["prompt_token_num"]
    initial_stage_completion_token_sum += doc["completion_token_num"]

print(f"Initial stage prompt token sum: {initial_stage_prompt_token_sum}")
print(f"Completion token sum: {initial_stage_completion_token_sum}")
print(f"Total token sum: {initial_stage_prompt_token_sum + initial_stage_completion_token_sum}")

In [ ]:
# Sum up completion and prompt tokens across all specified collections

collections = ["triplets", "filtered_triplets", "ontology_filtered_triplets"]

total_completion_token_num = 0
total_prompt_token_num = 0

for collection_name in collections:
    collection = db.get_collection(collection_name)
    pipeline = [
        {
            "$group": {
                "_id": None,
                "sum_completion_token_num": {"$sum": "$completion_token_num"},
                "sum_prompt_token_num": {"$sum": "$prompt_token_num"}
            }
        }
    ]
    result = list(collection.aggregate(pipeline))
    if result:
        total_completion_token_num += result[0].get("sum_completion_token_num", 0)
        total_prompt_token_num += result[0].get("sum_prompt_token_num", 0)

print(f"Total sum of completion_token_num across all collections: {total_completion_token_num}")
print(f"Total sum of prompt_token_num across all collections: {total_prompt_token_num}")
print(f"Grand total token sum: {total_completion_token_num + total_prompt_token_num}")

print(f"All stage prompt token sum: {initial_stage_prompt_token_sum + total_prompt_token_num}")
print(f"All stage completion token sum: {initial_stage_completion_token_sum + total_completion_token_num}")
print(f"Total token sum: {initial_stage_prompt_token_sum + initial_stage_completion_token_sum + total_prompt_token_num + total_completion_token_num}")


